### Индивидуальное задание. Линейные ОДУ второго порядка с постоянными коэффициентами

#### Решить задачу Коши аналитически, методом Эйлера и методом Рунге-Кутты. Построить в одной координатной плоскости графики точного и приближенных решений. Шаг принять равным 0.01

In [102]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import os

In [103]:
%run document_generator.ipynb

In [104]:
x, y, C = sp.symbols("x y C", real=True)

In [105]:
"""
Returns coefficients (a, b, c) for a random "derivative function" in the form d/dx y(x) = (a*y(x) + b) * x**c, 
where c is N+ or 1/(N+) and !(a == 0 and b == 0).
"""
def generate_coefficients() -> tuple[sp.Integer, sp.Integer, sp.Rational]:
    def coef(hi, low = 0):
        return sp.sympify(np.random.choice([-1, 1]) * np.random.randint(low, hi + 1))

    while True:
        a = coef(10)
        b = coef(10)
        if a == 0 and b == 0:
            continue

        c_denom = sp.sympify(np.random.randint(1, 21))
        c_num = sp.sympify(np.random.randint(1, 4*c_denom + 1))
        c = c_denom / c_num

        return a, b, c

In [106]:
"""
Returns an initial point in the form (0, x) for Cauchy problem.  
"""
def generate_initial_point() -> sp.Point2D:
    return sp.Point2D(0, np.random.choice([-1, 1]) * np.random.randint(11))

In [107]:
"""
Analytically solves Cauchy problem of function in the form d/dx y(x) = (a*y(x) + b) * x**c,
with a, b, c coefficients and initial point given as parameters.
Returns the resulting function if a solution exists; otherwise, returns None
"""
def solve_analytically(a: sp.Number, b: sp.Number, c: sp.Number, initial_p: sp.Point2D) -> sp.Expr:
    lhs = sp.integrate(sp.S(1) / (a*y + b), y)
    rhs = sp.integrate(x**c, x) + C

    lhs_subbed = lhs.subs({x: initial_p[0], y: initial_p[1]})
    rhs_subbed = rhs.subs({x: initial_p[0], y: initial_p[1]})

    c_values = sp.solveset(sp.Eq(lhs_subbed, rhs_subbed), C)
    if len(c_values) == 0:
        return None
    
    c_value, = c_values
    rhs = rhs.subs({C: c_value})

    eqs = sp.solve(sp.Eq(lhs, rhs), y)
    assert len(eqs) > 0, "Could not solve for y"

    return eqs[0]

In [108]:
APPROX_STEP = 0.01
APPROX_RANGE = 200

In [109]:
def solve_euler(y_deriv: sp.Expr, initial_p: sp.Point2D) -> list[tuple[float, float]]:
    x_k, y_k = initial_p
    res = [initial_p]

    for i in range(APPROX_RANGE):
        f_k = y_deriv.subs({x: x_k, y: y_k})

        x_k += APPROX_STEP
        y_k += APPROX_STEP*float(f_k)

        res.append((float(x_k), float(y_k)))

    return res

In [110]:
def solve_runge_kutta(y_deriv: sp.Expr, initial_p: sp.Expr) -> list[tuple[float, float]]:
    x_k, y_k = initial_p
    res = [initial_p]

    for i in range(APPROX_RANGE):
        f_k = y_deriv.subs({x: x_k, y: y_k})

        y_k1 = y_k + APPROX_STEP*f_k
        x_k += APPROX_STEP
        
        f_k1 = (f_k + y_deriv.subs({x: x_k, y: y_k1})) / sp.S(2)
        y_k += APPROX_STEP*f_k1
        
        res.append((float(x_k), float(y_k)))

    return res

In [111]:
PICS_PATH = "../out/answers/answer17/"
PLOT_X_LIMITS = (0, 2)

In [112]:
def create_plot(
        y_solved: sp.Expr, 
        euler_solution: list[tuple[float, float]], 
        runge_kutta_solution: list[tuple[float, float]],
        name: str):
    y_solved_lam = sp.lambdify(x, y_solved, modules=["numpy"])

    X_len = int((PLOT_X_LIMITS[1] - PLOT_X_LIMITS[0]) / APPROX_STEP)
    X = np.linspace(*PLOT_X_LIMITS, X_len)

    fig = plt.figure()

    plt.plot(X, y_solved_lam(X), color="red", label="Analytical")
    plt.plot(*zip(*euler_solution), color="green", label="Euler")
    plt.plot(*zip(*runge_kutta_solution), color="black", label="Runge-Kutta")

    plt.legend()

    os.makedirs(os.path.dirname(PICS_PATH), exist_ok=True)
    plt.savefig(PICS_PATH + name)
    
    # Close plot to save on resources.
    plt.close(fig)

In [113]:
DESCRIPTION = """Построить графики функций $f(x)$ и $g(x)$, уточнить координаты точек пересечения, решая численно соответствующее уравнение. 
На графике отметить и подписать буквами A1, A2, ... точки пересечения графиков, вывести в легенде формулы функций, подписать оси."""

TASK = """\\begin{{align*}}
    \\frac{{d}}{{dx}} y(x) = {y} && y({initial_p_x}) = {initial_p_y}
\\end{{align*}}"""

SOLUTION = """\\begin{{align*}}
    \\frac{{d}}{{dx}} y(x) = {y} && y({initial_p_x}) = {initial_p_y}
\\end{{align*}}
Analytical solution: $y(x) = {y_solved}$

\\includegraphics[scale=0.8]{{ {{ {plot_name} }} }}
"""

In [114]:
def variant_generator(variant_n: int):
    a, b, c = generate_coefficients()
    initial_p = generate_initial_point()
    analytic_solution = sp.Expr()

    while (analytic_solution := solve_analytically(a, b, c, initial_p)) == None:
        a, b, c = generate_coefficients()
        initial_p = generate_initial_point()

    y_deriv = (a*y + b) * x**c
    
    euler_solution = solve_euler(y_deriv, initial_p)
    runge_kutta_solution = solve_runge_kutta(y_deriv, initial_p)

    name = f"{variant_n}.png"
    create_plot(analytic_solution, euler_solution, runge_kutta_solution, name)

    y_f = sp.Function("y")
    y_deriv_subbed = y_deriv.subs({y: y_f(x)})
    if np.random.randint(2) == 0:
        y_deriv_subbed = sp.expand(y_deriv_subbed)

    y_latex = sp.latex(y_deriv_subbed)
    initial_p_x_latex = sp.latex(initial_p[0])
    initial_p_y_latex = sp.latex(initial_p[1])

    return TASK.format(y = y_latex, initial_p_x = initial_p_x_latex, initial_p_y = initial_p_y_latex), \
        SOLUTION.format(y = y_latex, initial_p_x = initial_p_x_latex, initial_p_y = initial_p_y_latex,
                        y_solved = sp.latex(analytic_solution), plot_name=name)

DOC = DocumentGenerator(variant_generator, DESCRIPTION)

In [115]:
write_tasks_and_solutions(DOC, "../out/tasks/task-17.tex", "../out/answers/answer17/answer-17.tex", 150)